# Boyce-Codd Normal Form (BCNF)

You can think of BCNF as a stricter version of **Third Normal Form (3NF)**. It makes sure that **every piece of information in a table depends only on a “true” unique identifier**—what we call a **candidate key**.

That is, if you have a table with a composite key (a key made up of two or more columns), BCNF ensures that all the other columns depend on the entire composite key, not just part of it.

## Table violating BCNF

Imagine a table that stores course registrations:

| StudentID | CourseCode | Instructor |
|-----------|------------|------------|
| S1        | CS101      | Prof. Kim  |
| S2        | CS101      | Prof. Kim  |
| S3        | CS102      | Prof. Jones|

This table seems fine. But notice:

* The **instructor name is repeated** for every student in the same course.
* What if someone **changes one instance** of the instructor to “Dr. Kim” but forgets to update the others?
* Or if **we delete the last student from CS101**, we might lose the information about the instructor for that course.

These are **update and deletion anomalies**. BCNF helps avoid them.

## 🧠 When is a table in BCNF?

A table is in **BCNF** if:

For every **functional dependency** in the table, the **left-hand side** is a candidate key.

In other words:

If some column(s) determine other columns, then those column(s) must uniquely identify a row in the table.

## Another Table Violating BCNF

| CourseID | Instructor  | Classroom |
|----------|-------------|-----------|
| 1        | Dr. Smith  | Room 101  |
| 2        | Dr. Smith  | Room 102  |
| 3        | Dr. Johnson| Room 101  |

**Why it violates BCNF:** The "Instructor" column determines "Classroom", but "Instructor" is not a candidate key.

In [1]:
# Import SQLite library
import sqlite3

# Create an in-memory SQLite database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

In [2]:
# Create a table that is not in BCNF
cursor.execute('''
CREATE TABLE Courses (
    CourseID INTEGER PRIMARY KEY,
    Instructor TEXT,
    Classroom TEXT
)''')

# Insert data
cursor.executemany('INSERT INTO Courses (CourseID, Instructor, Classroom) VALUES (?, ?, ?)', [
    (1, 'Dr. Smith', 'Room 101'),
    (2, 'Dr. Smith', 'Room 102'),
    (3, 'Dr. Johnson', 'Room 101')
])

# Query the table
cursor.execute('SELECT * FROM Courses')
for row in cursor.fetchall():
    print(row)

(1, 'Dr. Smith', 'Room 101')
(2, 'Dr. Smith', 'Room 102')
(3, 'Dr. Johnson', 'Room 101')


## Decomposing into BCNF

To convert the table into BCNF, we decompose it into two tables: one for instructors and classrooms, and another for courses and instructors.

In [3]:
# Create tables in BCNF
cursor.execute('''
CREATE TABLE Instructors (
    Instructor TEXT,
    Classroom TEXT,
    PRIMARY KEY (Instructor, Classroom)
)''')

cursor.execute('''
CREATE TABLE CourseAssignments (
    CourseID INTEGER,
    Instructor TEXT,
    PRIMARY KEY (CourseID, Instructor),
    FOREIGN KEY (Instructor) REFERENCES Instructors(Instructor)
)''')

# Insert data into BCNF tables
cursor.executemany('INSERT INTO Instructors (Instructor, Classroom) VALUES (?, ?)', [
    ('Dr. Smith', 'Room 101'),
    ('Dr. Smith', 'Room 102'),
    ('Dr. Johnson', 'Room 101')
])

cursor.executemany('INSERT INTO CourseAssignments (CourseID, Instructor) VALUES (?, ?)', [
    (1, 'Dr. Smith'),
    (2, 'Dr. Smith'),
    (3, 'Dr. Johnson')
])

# Query the BCNF tables
print('Instructors Table:')
cursor.execute('SELECT * FROM Instructors')
for row in cursor.fetchall():
    print(row)

print('\nCourseAssignments Table:')
cursor.execute('SELECT * FROM CourseAssignments')
for row in cursor.fetchall():
    print(row)

Instructors Table:
('Dr. Smith', 'Room 101')
('Dr. Smith', 'Room 102')
('Dr. Johnson', 'Room 101')

CourseAssignments Table:
(1, 'Dr. Smith')
(2, 'Dr. Smith')
(3, 'Dr. Johnson')


## 💡 Summary

* BCNF is about making sure only candidate keys determine other columns.
* It prevents data anomalies by keeping your design clean.
* If a table has a dependency where a non-candidate key determines other data, it’s not in BCNF, and you should split it.